In [210]:
import importlib

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import Module as m
importlib.reload(m) 

<module 'Module' from 'c:\\Users\\TUF\\OneDrive\\Documents\\Code\\Vs Code\\Python_DataWarehouse\\Layers\\Module.py'>

In [6]:
cust = pd.read_csv("../Dataset/CRM/customers.csv.gz")
orde = pd.read_csv("../Dataset/CRM/orders.csv.gz")
menu = pd.read_csv("../Dataset/CRM/menu_items.csv.gz")
ord_itm = pd.read_csv("../Dataset/CRM/order_items.csv.gz")
res = pd.read_csv("../Dataset/CRM/restaurants.csv.gz")
deli = pd.read_csv("../Dataset/CRM/delivery_partners.csv.gz")

In [ ]:
emp = pd.read_csv("../Dataset/ERP/employees.csv.gz")
inv = pd.read_csv("../Dataset/ERP/inventory.csv.gz")
kic = pd.read_csv("../Dataset/ERP/kitchen_logs.csv.gz")
sup_itm = pd.read_csv("../Dataset/ERP/supplier_items.csv.gz")
sup = pd.read_csv("../Dataset/ERP/suppliers.csv.gz")
chf = pd.read_csv("../Dataset/ERP/chefs.csv.gz")

### CRM Tables

In [92]:
ord_itm.head(5)

,order_item_id,order_id,item_id,quantity,unit_price,line_total
0,OI0000001,O0127050,M0344,3,554.91,1664.73
1,OI0000002,O0139788,M0103,3,114.8,344.4
2,OI0000003,O0438146,NaN,3,750.97,NaN
3,OI0000004,O0332091,M0496,1,443.56,NaN
4,OI0000005,O0435104,M0354,5,544.06,2720.3


In [353]:
order_item = pd.DataFrame()
# order_item_id
ord_itm_id = ord_itm['order_item_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
ord_itm_id1 = ['OI0'+i[2::] if len(i) == 8 else i for i in ord_itm_id]
order_item['order_item_id'] = [i if len(i) == 9 else None for i in ord_itm_id1]
# order_id
order_id = ord_itm['order_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
order_id1 = ['O0'+i[1::] if len(i) == 7 else i for i in order_id]
order_id2 = [ i if len(i) == 8 else None for i in order_id1]
order = pd.read_pickle("../Layers/Silver/CRM/order.pkl")
order_id3 = set(order['order_id'])
order_item['order_id'] = [i if i in order_id3 else None for i in order_id]
# item_id
item_id = ord_itm['item_id'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning(x)))
item_id1 = [i[0]+'0'+i[1::] if len(i) == 4 else i if len(i) == 5 else None for i in item_id]
menu_item = pd.read_pickle("../Layers/Silver/CRM/menu_item.pkl")
item_id2 = set(menu_item['item_id'])
order_item['item_id'] = [None if i not in item_id2 else i for i in item_id1]
# quantity
qty = ord_itm['quantity'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning(x)))
order_item['quantity'] = [0 if i in ('nan', 'na', '0') or len(i) > 2 else i for i in qty]
# unit_price
price = ord_itm['unit_price'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning_d(x)))
order_item['price'] = [0 if i in ('nan','na','n') or float(i) <= 0 else float(i) for i in price]
# line_total
price = ord_itm['line_total'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning_d(x)))
order_item['line_total'] = [0 if i in ('nan','na','n') or float(i) <= 0 else float(i) for i in price]

order_item.dropna(subset = 'order_item_id').drop_duplicates(subset = 'order_item_id').sort_values(by = 'order_item_id').reset_index().drop('index', axis = 1)

,order_item_id,order_id,item_id,quantity,price,line_total
0,OI0000001,O0127050,M0344,3,554.91,1664.73
1,OI0000002,O0139788,M0103,3,114.80,344.40
2,OI0000003,O0438146,None,3,750.97,0.00
3,OI0000004,O0332091,M0496,1,443.56,0.00
4,OI0000005,O0435104,M0354,5,544.06,2720.30
...,...,...,...,...,...,...
974313,OI0999996,O0286240,M0274,2,56.93,113.86
974314,OI0999997,O0489082,M0404,3,692.35,2077.05
974315,OI0999998,None,M0306,4,162.90,651.60
974316,OI0999999,O0063126,M0347,4,427.81,1711.24


In [332]:
b = [len(i) for i in price]

In [333]:
id = pd.DataFrame({'id':price,'len':b})

In [342]:
id[id['len'] == 8]

,id,len
